In [ ]:
!git clone https://github.com/anonymousNeurIPS2021submission5254/SupplementaryMaterial.git

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
from time import time
import gc
import copy
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
sys.path.append('/kaggle/working/SupplementaryMaterial')
from MLRNN import MLRNNClassifier

# Data Prep

In [ ]:
train = pd.read_feather('/kaggle/input/september-feather/train_rg')
train_nan_count = pd.read_feather('/kaggle/input/september-feather/train_rg_min', columns = ['nan_count'])
test =  pd.read_feather('/kaggle/input/september-feather/test_rg')
test_nan_count = pd.read_feather('/kaggle/input/september-feather/test_rg_min', columns = ['nan_count'])

ss = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')
FEATURES = [feat for feat in train.columns if 'f' in feat]
TARGET = 'claim'

######################
#Feature Engineering
#
#Rank Gaussed features with row-wise statistics and 
# count of nans per row
######################

#Adding row wise features
ROW_WISE = ['min','max','std','var','mean']
for df in [train, test]:
    df['min'] = df[FEATURES].min(axis=1)
    df['max'] = df[FEATURES].max(axis=1)
    df['std'] = df[FEATURES].std(axis=1)
    df['var'] = df[FEATURES].var(axis=1)
    df['mean'] = df[FEATURES].mean(axis=1)

#Adding count all all nas
train['nan_count'] = train_nan_count['nan_count'] / 14
test['nan_count'] = test_nan_count['nan_count'] / 14

#Final Features for nn
FEATURES = FEATURES + ROW_WISE + ['nan_count']

# Training

We pick the best 5 networks from 30 trained networks based on validation performance.

Copied from [the paper's repo](https://github.com/anonymousNeurIPS2021submission5254/SupplementaryMaterial/blob/main/MLR%20NN.ipynb).

In [ ]:
#Getting Parameter list
common_parameters = {"max_runtime" : 900, "width" : 1024}
#from https://www.kaggle.com/ottpocket/optuna-muddling?scriptVersionId=75812644
MLR1_parameters = copy.deepcopy(common_parameters)
MLR1_parameters["depth"] = 3
MLR1_parameters["learning_rate"] = 0.0008981541684340913
MLR1_parameters["max_iter"] = 786

In [ ]:
rep = 20 #number of NN
selected = 5
architectures = [MLR1_parameters]


predictions = []
valid_performances = []
for i, rs in enumerate(range(rep)):
    start = time()
    print(f'Network: {i+1} of {rep},', sep=' ')
    model = MLRNNClassifier(random_state = rs, **MLR1_parameters)
    predictions.append(model.fit(train[FEATURES].values, train[TARGET].values).predict_proba(test[FEATURES].values)[:,1].astype(np.float16))
    valid_performances.append(np.max(model.record["validation"]))
    model.delete_model_weights()
    del model
    print(f'Val: {valid_performances[-1] :.4f}, time: {time() - start :.2f}')
ss[TARGET] = np.mean([predictions[index] for index in np.argsort(valid_performances)[-selected:]],axis = 0)


In [ ]:
ss.to_csv('ss.csv', index=False)